GROUp A - ALDO

In [ ]:
import mne
from mne.preprocessing import ICA
import matplotlib.pyplot as plt
import numpy as np
import os
import glob

# --- Instalasi (Jalankan ini hanya jika pustaka belum terinstal) ---
# Jika Anda belum memiliki MNE, scikit-learn, atau pyedflib, hapus tanda '#' di bawah ini
# dan jalankan di terminal atau command prompt:
# pip install mne scikit-learn pyedflib

# --- Langkah 1: Memuat Data EDF dari Folder ---
# Tentukan jalur ke folder yang berisi file-file EDF.
edf_folder_paths = "Group A\Aldo\EEG18EDFCase"
try:
    # Mencari semua file .EDF di dalam folder
    edf_files = glob.glob(os.path.join(edf_folder_paths, "*.EDF"))
    
    # Memeriksa jika ada file yang ditemukan
    if not edf_files:
        print(f"Tidak ada file EDF ditemukan di: '{edf_folder_paths}'. Mohon periksa kembali jalurnya.")
    else:
        # Looping melalui setiap file EDF yang ditemukan
        for file_path in edf_files:
            edf_file_path = file_path

            try:
                # Membaca data EDF. preload=True memuat data ke memori.
                print(f"Membaca file: {edf_file_path}...")
                raw = mne.io.read_raw_edf(edf_file_path, preload=True)
                raw.set_montage('standard_1020', on_missing='ignore')
            
            except FileNotFoundError:
                print(f"File '{edf_file_path}' tidak ditemukan.")
                print("Membuat data EEG dummy untuk demonstrasi.")
                
                # Membuat data dummy untuk demonstrasi jika file tidak ada
                sfreq = 256  # Frekuensi sampling dalam Hz
                ch_names = ['Fp1', 'Fp2', 'F7', 'F8', 'T7', 'T8', 'P7', 'P8', 'O1', 'O2']
                n_channels = len(ch_names)
                duration_s = 60  # Durasi dalam detik
                
                # Membuat sinyal sinus 10 Hz untuk simulasi kedipan mata
                t = np.arange(sfreq * duration_s) / sfreq
                eye_blink = np.sin(2 * np.pi * 10 * t) * 50e-6
                
                # Membuat data acak untuk simulasi aktivitas otak
                eeg_data = np.random.randn(n_channels, len(t)) * 10e-6
                
                # Menambahkan sinyal kedipan mata ke beberapa saluran
                eeg_data[0] += eye_blink  # Menambahkan kedipan mata ke Fp1
                eeg_data[1] += eye_blink  # Menambahkan kedipan mata ke Fp2

                info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types='eeg')
                raw = mne.io.RawArray(eeg_data, info)
                raw.set_montage('standard_1020')

            # --- Langkah 2: Analisis Spektrum dengan FFT ---
            print("Menganalisis spektrum frekuensi dengan FFT...")
            raw.compute_psd().plot()
            plt.title(f'Power Spectral Density (PSD) - {os.path.basename(edf_file_path)}')
            plt.show()

            # --- Langkah 3: Pra-pemrosesan Data ---
            print("Melakukan pra-pemrosesan data...")
            raw.filter(1., 40., fir_design='firwin')
            raw.set_eeg_reference('average', projection=True)

            # --- Langkah 4: Menyiapkan dan Menjalankan ICA ---
            print("Menjalankan ICA...")
            ica = ICA(n_components=5, random_state=97, method='fastica')
            ica.fit(raw)

            # --- Langkah 5: Visualisasi Hasil ICA ---
            print("Membuat plot komponen ICA...")
            ica.plot_sources(raw, show_scrollbars=False)
            plt.show()

# Handling error
except FileNotFoundError:
    print(f"Folder '{edf_folder_paths}' tidak ditemukan. Mohon periksa kembali jalurnya.")